## Data Extraction

In [12]:
import pandas as pd
import os
from src.data_extraction.data_extraction import extract_initial_data

In [13]:
df_assets, df_ine, df_osm, df_pois, df_polygons = extract_initial_data(
    root_dir="data"
)